In [43]:

from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE
import pandas as pd
import sklearn
from imblearn.combine import SMOTETomek
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score,confusion_matrix,accuracy_score,classification_report
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.preprocessing import PolynomialFeatures,StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
pd.set_option('display.max_columns', 500)
import pickle as pkl

In [2]:
df = pd.read_csv('polydf.csv',index_col='Unnamed: 0')
df1 = pd.read_csv('prepeddata.csv',index_col='Unnamed: 0')
df1['stimulant'] = df1['stimulant'].apply(lambda x: 'low' if x <= 2 else 'high')
df
RandomForestClassifier()


RandomForestClassifier()

In [3]:
stimtestdata = df1.drop(['hallucinagen','depressant'],axis = 1)
stimx = df
stimy = df1.stimulant
stimx

sm = SMOTE()
tl = TomekLinks()
sampling = SMOTETomek(sampling_strategy='auto', random_state=None, smote=sm, tomek=tl, n_jobs=1)

stimx, stimy = sampling.fit_resample(stimx, stimy)



In [4]:
stimXtrain,stimXtest,stimYtrain,stimYtest = train_test_split(stimx,stimy)

In [5]:
stimscaler = StandardScaler()
stimXtrain = stimscaler.fit_transform(stimXtrain)
stimXtest = stimscaler.transform(stimXtest)
stimYtrain

138      low
1406     low
1078     low
3024    high
2497    high
        ... 
1797     low
1391     low
1921    high
2337    high
840     high
Name: stimulant, Length: 2431, dtype: object

Logistic regression classifiers

In [6]:
# mod = LogisticRegression(class_weight = 'balanced'
#                          ,max_iter=1000,solver='liblinear',warm_start=True,tol=0.2)
# mod.fit(stimXtrain,stimYtrain)
# stimtrianlogypred = mod.predict(stimXtrain)
# stimtestlogypred = mod.predict(stimXtest)

In [7]:
logregparam_grid = {'C': [0.001,0.1,1, 10],'max_iter' :[1000],'class_weight':['auto'],'warm_start' :[True,False], 'solver': ['liblinear', 'rbf','sag']}
mod = GridSearchCV(LogisticRegression(), logregparam_grid, n_jobs=-1, cv=5, verbose=1)
mod.fit(stimXtrain,stimYtrain)
stimtrianlogypred = mod.predict(stimXtrain)
stimtestlogypred = mod.predict(stimXtest)


Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    5.0s finished
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


In [8]:
polymodel1 = mod.best_estimator_


In [9]:
f1_score(stimYtrain,stimtrianlogypred,average='weighted'),f1_score(stimYtest,stimtestlogypred,average='weighted')

(0.7442059427299474, 0.7038281949919063)

In [10]:
logregparam_grid2 = {'tol': [0,0.0001,0.001,0.1,1, 10]
                     ,'C': [0,0.001,0.1,1, 10]
                     ,'penalty': ['l1','l2','elasticnet']
                     ,'max_iter' :[1000],'class_weight':['auto']
                     ,'warm_start' :[True,False]
                     , 'solver': ['lbfgs','liblinear', 'rbf','sag','elasticnet','saga',]
                     ,'dual':[False,True],"l1_ratio":[None,0.01,0.1,0.2,0.3,0.4]
                    ,'fit_intercept':[True,False]}
mod2 = GridSearchCV(LogisticRegression(), logregparam_grid2, n_jobs=-1, cv=5, verbose=1)
mod2.fit(stimXtrain,stimYtrain)
stimtrianlog2ypred = mod2.predict(stimXtrain)
stimtestlog2ypred = mod2.predict(stimXtest)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 25920 candidates, totalling 129600 fits


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 3560 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 19560 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done 31580 tasks      | elapsed:   35.7s
[Parallel(n_jobs=-1)]: Done 34652 tasks      | elapsed:   52.4s
[Parallel(n_jobs=-1)]: Done 38849 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 56322 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 60884 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 73168 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 84232 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 89972 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 107598 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 112942 tasks      | elapsed:  7.9min
[Parallel(n_jobs=-1)]: Done 128168 tasks      | elapsed:  9.0min
[Parallel(n_jobs=-1)]: Done 129600 out of 129600 | elapsed:  9.1min finished


In [16]:
f1_score(stimYtrain,stimtrianlog2ypred,average='weighted'),f1_score(stimYtest,stimtestlog2ypred,average='weighted')
polymodel2 = mod2.best_estimator_

In [17]:
logregparam_grid3 = {'tol': [0,0.0001,0.001,0.1,1, 10]
                     ,'penalty': ['l1','l2','elasticnet']
                     ,'max_iter' :[4000],'class_weight':['auto']
                     ,'warm_start' :[True]
                     , 'solver': ['lbfgs','liblinear', 'rbf','sag','elasticnet','saga']
                    }
mod3 = GridSearchCV(LogisticRegression(), logregparam_grid3, n_jobs=-1, cv=5, verbose=1)
mod3.fit(stimXtrain,stimYtrain)
stimtrianlog3ypred = mod3.predict(stimXtrain)
stimtestlog3ypred = mod3.predict(stimXtest)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 393 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:   15.5s finished


In [18]:
f1_score(stimYtrain,stimtrianlog3ypred,average='weighted'),f1_score(stimYtest,stimtestlog3ypred,average='weighted')

(0.7427195188532965, 0.7024282442165305)

In [19]:
polymodel3 = mod3.best_estimator_
#('polymodel1',polymodel1),('polymodel1',polymodel2),('polymodel1',polymodel3)

In [20]:
pkl.dump(polymodel1,open('PolyModel1.pkl','wb'))
pkl.dump(polymodel2,open('PolyModel2.pkl','wb'))
pkl.dump(polymodel3,open('PolyModel3.pkl','wb'))

Decision Trees

In [21]:
# dtc = DecisionTreeClassifier(criterion='gini',ccp_alpha=0.0055)
# dtc.fit(stimXtrain,stimYtrain)
# dtcytrainpred = dtc.predict(stimXtrain)
# dtcytestpred = dtc.predict(stimXtest)
parameters={'max_depth': range(1,20,1),'ccp_alpha':[x/60 for x in list(range(0,60,1))],'max_features':range(1,30,1),'criterion':['gini']}
dtc = GridSearchCV(DecisionTreeClassifier(),param_grid=parameters,verbose=True,n_jobs=-1)
dtc.fit(stimXtrain,stimYtrain)
dtcytrainpred = dtc.predict(stimXtrain)
dtcytestpred = dtc.predict(stimXtest)

Fitting 5 folds for each of 33060 candidates, totalling 165300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 1640 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 5640 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done 11240 tasks      | elapsed:   19.3s
[Parallel(n_jobs=-1)]: Done 18440 tasks      | elapsed:   32.0s
[Parallel(n_jobs=-1)]: Done 27240 tasks      | elapsed:   47.7s
[Parallel(n_jobs=-1)]: Done 37640 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 49640 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 63240 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 78440 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 95240 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 113640 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 133640 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 155240 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 

In [22]:
f1_score(stimYtrain,dtcytrainpred,average='weighted'),f1_score(stimYtest,dtcytestpred,average='weighted')

(1.0, 0.8120201090536966)

In [23]:
confusion_matrix(stimYtest,dtcytestpred)

array([[349,  51],
       [101, 310]])

In [24]:
parameters={'criterion':['entropy','gini'],'min_samples_leaf':range(0,20,2)}
dtc2 = GridSearchCV(DecisionTreeClassifier(),param_grid=parameters,verbose=True,n_jobs=-1)
dtc2.fit(stimXtrain,stimYtrain)
dtcytrainpred = dtc2.predict(stimXtrain)
dtcytestpred = dtc2.predict(stimXtest)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.5s finished


In [25]:
f1_score(stimYtrain,dtcytrainpred,average='weighted'),f1_score(stimYtest,dtcytestpred,average='weighted')

(0.9835418882676786, 0.8530527353914533)

In [26]:
confusion_matrix(stimYtest,dtcytestpred)

array([[359,  41],
       [ 78, 333]])

In [27]:
# dtc3 = DecisionTreeClassifier(class_weight='balanced',criterion='gini',ccp_alpha=0.006599999)
# dtc3.fit(stimXtrain,stimYtrain)
# dtc3ytrainpred = dtc3.predict(stimXtrain)
# dtc3ytestpred = dtc3.predict(stimXtest)
parameters={'min_impurity_decrease':[0+x/100 for x in range(1,10)],'ccp_alpha':[x/60 for x in list(range(0,100,1))],'criterion':['entropy','gini'],'min_samples_leaf':range(0,20,2)}
dtc3 = GridSearchCV(DecisionTreeClassifier(),param_grid=parameters,verbose=True,n_jobs=-1)
dtc3.fit(stimXtrain,stimYtrain)
dtcytrainpred = dtc3.predict(stimXtrain)
dtcytestpred = dtc3.predict(stimXtest)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 18000 candidates, totalling 90000 fits


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 1640 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 5640 tasks      | elapsed:   10.2s
[Parallel(n_jobs=-1)]: Done 11240 tasks      | elapsed:   20.3s
[Parallel(n_jobs=-1)]: Done 18440 tasks      | elapsed:   33.0s
[Parallel(n_jobs=-1)]: Done 27240 tasks      | elapsed:   48.4s
[Parallel(n_jobs=-1)]: Done 37640 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 49640 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 63240 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 78440 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 90000 out of 90000 | elapsed:  2.6min finished


In [28]:
f1_score(stimYtrain,dtcytrainpred,average='weighted'),f1_score(stimYtest,dtcytestpred,average='weighted')


(0.778696090537862, 0.7902705919422134)

In [29]:
polymodel4 = dtc.best_estimator_
polymodel5 = dtc2.best_estimator_
polymodel6 = dtc3.best_estimator_

pkl.dump(polymodel4,open('PolyModel4.pkl','wb'))
pkl.dump(polymodel5,open('PolyModel5.pkl','wb'))
pkl.dump(polymodel6,open('PolyModel6.pkl','wb'))
#('polymodel1',polymodel1),('polymodel1',polymodel2),('polymodel1',polymodel3),('polymodel4',polymodel4),('polymodel5',polymodel5),('polymodel6',polymodel6)


Knearest

In [30]:
parameters={'n_neighbors':range(1,50)}
knn = GridSearchCV(KNeighborsClassifier(),param_grid=parameters,verbose=True,n_jobs=-1)
knn.fit(stimXtrain,stimYtrain)
knnytrainpred = knn.predict(stimXtrain)
knnytestpred = knn.predict(stimXtest)


Fitting 5 folds for each of 49 candidates, totalling 245 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 222 out of 245 | elapsed:    2.6s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done 245 out of 245 | elapsed:    2.7s finished


In [31]:
parameters={'n_neighbors':range(50,1130)}
knn2 = GridSearchCV(KNeighborsClassifier(),param_grid=parameters,verbose=True,n_jobs=-1)
knn2.fit(stimXtrain,stimYtrain)
knn2ytrainpred = knn2.predict(stimXtrain)
knn2ytestpred = knn2.predict(stimXtest)


Fitting 5 folds for each of 1080 candidates, totalling 5400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 828 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done 1528 tasks      | elapsed:   24.2s
[Parallel(n_jobs=-1)]: Done 2428 tasks      | elapsed:   43.2s
[Parallel(n_jobs=-1)]: Done 3528 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 4828 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 5400 out of 5400 | elapsed:  1.7min finished


In [32]:
polymodel7 = knn.best_estimator_
polymodel8 = knn2.best_estimator_

#('polymodel1',polymodel1),('polymodel1',polymodel2),('polymodel1',polymodel3),('polymodel4',polymodel4),('polymodel5',polymodel5),('polymodel6',polymodel6),('polymodel7',polymodel7),('polymodel8',polymodel8)
pkl.dump(polymodel7,open('PolyModel7.pkl','wb'))
pkl.dump(polymodel8,open('PolyModel8.pkl','wb'))


Random Forest

In [33]:
parameters={'warm_start':[True,False],'class_weight':['balanced'],'criterion':['gini','entropy'],'n_estimators':[20,50,100,500]}
rfc = GridSearchCV(RandomForestClassifier(),param_grid=parameters,verbose=True,n_jobs=-1)
rfc.fit(stimXtrain,stimYtrain)
rfcytrainpred = rfc.predict(stimXtrain)
rfcytestpred = rfc.predict(stimXtest)


Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:   12.2s finished
/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:586: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn('class_weight presets "balanced" or '


In [34]:
rfctrainypred = rfc.predict(stimXtrain)
rfctestypred = rfc.predict(stimXtest)
f1_score(rfctrainypred,stimYtrain,average='weighted'),f1_score(rfctestypred,stimYtest,average='weighted')
confusion_matrix(stimYtest,rfctestypred)




array([[373,  27],
       [ 41, 370]])

In [35]:
parameters={'min_samples_leaf' : [3,5,7],'class_weight':['balanced'],'random_state':[True]
                         ,'criterion':['gini']
                         }
rfc2 = GridSearchCV(RandomForestClassifier(),param_grid=parameters,verbose=True,n_jobs=-1)
rfc2.fit(stimXtrain,stimYtrain)
rfc2ytrainpred = rfc2.predict(stimXtrain)
rfc2ytestpred = rfc2.predict(stimXtest)


Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:    0.7s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    1.1s finished


In [36]:
rfc2trainypred = rfc2.predict(stimXtrain)
rfc2testypred = rfc2.predict(stimXtest)
f1_score(rfc2trainypred,stimYtrain,average='weighted'),f1_score(rfc2testypred,stimYtest,average='weighted')



(0.9884822035875231, 0.9050774309629435)

In [37]:
polymodel9 = rfc.best_estimator_
polymodel10 = rfc2.best_estimator_
#('polymodel1',polymodel1),('polymodel1',polymodel2),('polymodel1',polymodel3),('polymodel4',polymodel4),('polymodel5',polymodel5),('polymodel6',polymodel6),('polymodel7',polymodel7),('polymodel8',polymodel8),('polymodel9',polymodel9),('polymodel10',polymodel10)

pkl.dump(polymodel9,open('PolyModel9.pkl','wb'))
pkl.dump(polymodel10,open('PolyModel10.pkl','wb'))

voting classifier

In [56]:

vtc = VotingClassifier([('polymodel1',polymodel1),('polymodel2',polymod2),('polymodel3',polymodel3)
                        ,('polymodel5',polymodel5),('polymodel6',polymodel6)
                        ,('polymodel7',polymodel7),('polymodel8',polymodel8),('polymodel9',polymodel9)
                        ,('polymodel10',polymodel10)],voting='soft',verbose = True,n_jobs=-1)
vtc.fit(stimXtrain,stimYtrain)

VotingClassifier(estimators=[('polymodel1',
                              LogisticRegression(C=10, class_weight='auto',
                                                 max_iter=1000, solver='sag',
                                                 warm_start=True)),
                             ('polymodel2',
                              LogisticRegression(C=1, class_weight='auto',
                                                 fit_intercept=False,
                                                 max_iter=1000, tol=1,
                                                 warm_start=True)),
                             ('polymodel3',
                              LogisticRegression(class_weight='auto',
                                                 max_iter=4000, penalty='l1',
                                                 solver='saga', tol=0.001...
                             ('polymodel7',
                              KNeighborsClassifier(n_neighbors=1)),
                         

In [57]:
vtcypred = vtc.predict(stimXtest)
vtcytrain = vtc.predict(stimXtrain)

print(f1_score(vtcytrain,stimYtrain,average='weighted'),f1_score(stimYtest,vtcypred,average='weighted'))


0.9671057737475964 0.8557575314432676


In [58]:
pkl.dump(vtc,open('vtcpoly.pkl','wb'))

In [62]:
confusion_matrix(vtcytrain,stimYtrain)

array([[1207,   60],
       [  20, 1144]])

In [60]:
print(classification_report(stimYtrain, vtcytrain))

              precision    recall  f1-score   support

        high       0.95      0.98      0.97      1227
         low       0.98      0.95      0.97      1204

    accuracy                           0.97      2431
   macro avg       0.97      0.97      0.97      2431
weighted avg       0.97      0.97      0.97      2431



In [61]:
print(classification_report(stimYtest, vtcypred))

              precision    recall  f1-score   support

        high       0.80      0.95      0.87       400
         low       0.95      0.76      0.84       411

    accuracy                           0.86       811
   macro avg       0.87      0.86      0.86       811
weighted avg       0.87      0.86      0.86       811

